## Lógica Computacional: 25/26
---
## TP1 - Ex2

$Grupo$ $05$ 

*   Vasco Ferreira Leite (A108399)
*   Gustavo da Silva Faria (A108575)
*   Afonso Henrique Cerqueira Leal (A108472)
---

Neste trabalho pretende-se generalizar o problema em várias direções:

- **Em primeiro lugar**, a grelha tem como parâmetro fundamental um inteiro que toma vários valores $n \in \{3,4,\dots\}$. Fundamentalmente a grelha passa de um quadrado com $n^{2} \times n^{2}$ células para um cubo tridimensional de dimensões $n^{2} \times n^{2} \times n^{2}$. Cada posição na grelha é representada por um triplo de inteiros $(i,j,k) \in \{1,\dots,n^{2}\}^{3}$.

- **Em segundo lugar**, as “regiões” que a definição menciona deixam de ser linhas, colunas e “sub-grids” para passar a ser qualquer “box” genérica com um número de células $\leq n^{3}$. Cada “box” é representada por um dicionário $D$ que associa, no estado inicial, cada posição $(i,j,k) \in D$ na “box” a um valor inteiro no intervalo $\{0,\dots,n^{3}\}$.

- Na inicialização da solução, as células associadas ao valor $0$ estão livres para ser instanciadas com qualquer valor não nulo. Se nessa fase, uma célula está associada a um valor não nulo, então esse valor está **fixo** e qualquer solução do problema não o modifica.

- A solução final do problema, tal como no problema original, verifica uma restrição do tipo *all-different* que, neste caso, tem a forma:  
  > Dentro de uma mesma “box”, todas as células têm valores distintos no intervalo  
  > $\{1,\dots,n^{3}\}$.

- Consideram-se neste problema duas formas básicas de “boxes”:
  - **“Cubos”** de $n^{3}$ células determinados pelo seu vértice superior, anterior, esquerdo.
  - **“Paths”** determinados pelo seu vértice de início, o vértice final e pela ordem entre os índices dos vértices sucessivos.

O **input** do problema é um conjunto de “boxes” e um conjunto de alocações de valores a células.


### Variáveis:

**Variáveis de decisão:**
- `cells[(i,j,k)]`: Valor da célula na posição $(i,j,k)$, domínio $[1, n^3]$

**Parâmetros:**
- `n`: Variável base do problema
- `size`: $n^2$ (dimensão de cada aresta do cubo)
- `domain_size`: $n^3$ tamanho do domínio (maior valor tomado por uma célula)
- `fixed_assignments`: Células com valores pré-definidos
- `cube_boxes`: Lista de cubos $n \times n \times n$
- `path_boxes`: Lista de paths entre vértices
- `generic_boxes`: Lista de boxes customizadas

In [1]:
from ortools.sat.python import cp_model

Função principal que cria o modelo CP-SAT, inicializa as variáveis para todas as células do cubo, adiciona os valores fixos fornecidos como input, aplica todas as restrições, e invoca o solver para encontrar uma solução.

In [2]:
def solve_sudoku_3d(n, valoresFixos=None, cube_boxes=None, generic_boxes=None):
    
    size = n * n
    domain_size = n * n * n
    
    model = cp_model.CpModel()
    solver = cp_model.CpSolver()
    
    cells = {}
    for i in range(1, size + 1):
        for j in range(1, size + 1):
            for k in range(1, size + 1):
                cells[(i, j, k)] = model.NewIntVar(1, domain_size, f'cell_{i}_{j}_{k}')
    
    if valoresFixos:
        for (i, j, k), value in valoresFixos.items():
            if value != 0:
                model.Add(cells[(i, j, k)] == value)
    
    _add_basic_constraints(model, cells, size)
    
    if cube_boxes:
        for start_i, start_j, start_k in cube_boxes:
            _add_cube_box(model, cells, n, size, start_i, start_j, start_k)
    
    if generic_boxes:
        for box_dict in generic_boxes:
            _add_generic_box(model, cells, box_dict)
    
    status = solver.Solve(model)
    
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        solution = {}
        for (i, j, k), var in cells.items():
            solution[(i, j, k)] = solver.Value(var)
        return solution
    else:
        print(f"Solução não encontrada. Status: {status}")
        return None

## Função: Restrições Básicas do Sudoku

Esta função implementa as restrições fundamentais do Sudoku.  
Para cada uma das três direções:

- **Linhas** (ao longo de `k`)
- **Colunas** (ao longo de `j`)
- **Pilhas** (ao longo de `i`)

Aplica a restrição **`AllDifferent`**, garantindo que não há valores repetidos em nenhuma dessas direções.


In [3]:
def _add_basic_constraints(model, cells, size):
    
    for i in range(1, size + 1):
        for j in range(1, size + 1):
            row_cells = [cells[(i, j, k)] for k in range(1, size + 1)]
            model.AddAllDifferent(row_cells)
    
    for i in range(1, size + 1):
        for k in range(1, size + 1):
            col_cells = [cells[(i, j, k)] for j in range(1, size + 1)]
            model.AddAllDifferent(col_cells)
    
    for j in range(1, size + 1):
        for k in range(1, size + 1):
            stack_cells = [cells[(i, j, k)] for i in range(1, size + 1)]
            model.AddAllDifferent(stack_cells)

## Função: Restrição box básica

Implementa boxes do tipo cubo, definidas pelo seu vértice superior-anterior-esquerdo. Constrói um subcubo de dimensões $n \times n \times n$ e aplica a restrição AllDifferent a todas as suas $n^3$ células.

In [4]:
def _add_cube_box(model, cells, n, size, start_i, start_j, start_k):
    
    cells_in_box = []
    
    for di in range(n):
        for dj in range(n):
            for dk in range(n):
                i = start_i + di
                j = start_j + dj  
                k = start_k + dk
                if 1 <= i <= size and 1 <= j <= size and 1 <= k <= size:
                    cells_in_box.append(cells[(i, j, k)])
    
    if len(cells_in_box) > 1:
        model.AddAllDifferent(cells_in_box)
    
    return cells_in_box

## Função: Restrição box custumizada

Implementa boxes genéricas customizáveis através de um dicionário. Permite definir qualquer conjunto de células como uma área e aplica all-diferent as mesmas, onde células com valor 0 são livres e devem ter valores diferentes entre si, e células com valores não-nulos têm esses valores fixados.

In [5]:
def _add_generic_box(model, cells, box_dict):

    free_cells = []
    
    for (i, j, k), value in box_dict.items():
        if value == 0:
            free_cells.append(cells[(i, j, k)])
        else:
            model.Add(cells[(i, j, k)] == value)
    
    if len(free_cells) > 1:
        model.AddAllDifferent(free_cells)
    
    return free_cells

## Função: Visualização da sulução

Função auxiliar para visualização da solução 3D. Esta função emprime o cubo em camadas ao longo de um eixo.

In [6]:
def print_solution_3d(solution, n):

    if not solution:
        print("Nenhuma solução para imprimir")
        return
    
    size = n * n
    
    coord_ranges = {
        'i': range(1, size + 1),
        'j': range(1, size + 1), 
        'k': range(1, size + 1)
    }
    
    fixed_coord = 'k'
    moving_coords = ['i', 'j']
    
    for fixed_val in coord_ranges[fixed_coord]:
        print(f"\n--- Camada {fixed_coord} = {fixed_val} ---")

        layer_matrix = []
        for mv1 in coord_ranges[moving_coords[0]]:
            row = []
            for mv2 in coord_ranges[moving_coords[1]]:
                coords = {fixed_coord: fixed_val, moving_coords[0]: mv1, moving_coords[1]: mv2}
                value = solution[(coords['i'], coords['j'], coords['k'])]
                row.append(value)
            layer_matrix.append(row)
        
        for row in layer_matrix:
            print(' '.join(f'{val:3d}' for val in row))

## Exemplo de problema
n = 5

In [ ]:
n = 4
size = n * n
domain_size = n * n * n
valoresFixos = {}   

valoresFixos[(1, 1, 1)] = 1
valoresFixos[(1, 1, 16)] = 16
valoresFixos[(1, 16, 1)] = 32
valoresFixos[(16, 1, 1)] = 48
valoresFixos[(16, 16, 16)] = 64  

center = size // 2  
quarter = size // 4  
    
valoresFixos[(center, center, center)] = domain_size // 2
valoresFixos[(1, center, center)] = 31
valoresFixos[(16, center, center)] = 42
valoresFixos[(center, 1, center)] = 44
valoresFixos[(center, 16, center)] = 27
valoresFixos[(center, center, 1)] = 56
valoresFixos[(center, center, 16)] = 38
valoresFixos[(quarter, quarter, quarter)] = 16
valoresFixos[(size-quarter, quarter, quarter)] = 41
valoresFixos[(quarter, size-quarter, quarter)] = 45
valoresFixos[(size-quarter, size-quarter, quarter)] = 11
    
cube_boxes = []
for start_i in range(1, size + 1, n):     
    for start_j in range(1, size + 1, n):
        for start_k in range(1, size + 1, n):
            cube_boxes.append((start_i, start_j, start_k))
    
generic_boxes = []
    
cruz_3d = {}
cruz_size = 3
half_cruz = cruz_size // 2
    
for offset in range(-half_cruz, half_cruz + 1):
    if center + offset <= size:
        cruz_3d[(center + offset, center, center)] = 0
    if center + offset <= size:
        cruz_3d[(center, center + offset, center)] = 0
    if center + offset <= size:
        cruz_3d[(center, center, center + offset)] = 0
    
generic_boxes.append(cruz_3d)

## Exemplo de problema
n = 2

In [12]:
n = 2  

valoresFixos = {
    (1, 1, 1): 1,
    (4, 4, 4): 8,
    (2, 2, 2): 4
}
    
cube_boxes = [
    (1, 1, 1),  
    (1, 3, 1),  
]
    
generic_boxes = [
    {
        (1, 2, 1): 0,  
        (1, 2, 2): 3,  
        (2, 2, 1): 0,    
        (2, 2, 2): 0   
    }
]

## Execução

In [22]:
solution = solve_sudoku_3d(
    n=n,
    valoresFixos=valoresFixos,
    cube_boxes=cube_boxes,
    generic_boxes=generic_boxes
)

if solution:
    print("\n✓ Solução encontrada!")
    print_solution_3d(solution, n)
    
    print("\n--- Verificação de valores fixos ---")
    for coords in [(1,1,1), (2,2,2), (4,4,4)]:
        print(f"Célula {coords}: {solution[coords]}")
else:
    print("\n✗ Não foi possível encontrar solução")

Solução não encontrada. Status: 0

✗ Não foi possível encontrar solução
